In [ ]:
# !pip install tensorflow-gpu==2.9.3
# # !pip list | grep tensorflow
# # !pip install tensorflow[and-cuda]
# !pip install torch
# !pip install nvidia-cudnn-cu11
!pip list | grep cud

In [ ]:
# # !pip uninstall kiwisolver matplotlib
# # !pip install kiwisolver matplotlib
# # !pip install tensorflow[and-cuda]==2.10
# # !pip list | grep tensor
# import tensorflow as tf
# print(tf.config.list_physical_devices('GPU'))
# # pip install 

In [ ]:
### Libraries ###
import os, sys, re, time, json, traceback, logging, datetime, gc, shutil, math, base64, pickle
from datetime import datetime
from pathlib import Path
import numpy as np
import torch
# from numba import cuda
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Audio Libraries ###
from pydub import AudioSegment, effects
# from pyannote.audio import Pipeline
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter
import soundfile as sf

In [ ]:
# print(torch.cuda.current_device())
# print(torch.cuda.is_available())
# print(torch.version.cuda)
# print(torch.backends.cudnn.version())
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
### Path of Files ###
input_video_filename_wo_ext = '1_min_video'
input_video_filename = 'Videos_Exampels/'+ input_video_filename_wo_ext +'.mp4' # The video files should be on mp4 format
output_video_path = 'Videos_Output/'

In [ ]:
# Create a new directory of output of videos
if not os.path.exists(output_video_path):
    os.makedirs(output_video_path)

# Create a new directory for processed audio if it does not exist
dir_output_audio = output_video_path + input_video_filename_wo_ext +'/Audio/'
if not os.path.exists(dir_output_audio):
    os.makedirs(dir_output_audio)
dir_output_audio_step1 = output_video_path + input_video_filename_wo_ext +'/Audio/Step1_ProcessAudio/'
if not os.path.exists(dir_output_audio_step1):
    os.makedirs(dir_output_audio_step1)
dir_output_audio_step2 = output_video_path + input_video_filename_wo_ext +'/Audio/Step2_Spleeter/'
if not os.path.exists(dir_output_audio_step2):
    os.makedirs(dir_output_audio_step2)
dir_output_audio_step3 = output_video_path + input_video_filename_wo_ext +'/Audio/Step3_Segments/'
if not os.path.exists(dir_output_audio_step3):
    os.makedirs(dir_output_audio_step3)
dir_output_audio_step4 = output_video_path + input_video_filename_wo_ext +'/Audio/Step4_Embeddings/'
if not os.path.exists(dir_output_audio_step4):
    os.makedirs(dir_output_audio_step4)
dir_output_audio_step5 = output_video_path + input_video_filename_wo_ext +'/Audio/Step5_Clusters/'
if not os.path.exists(dir_output_audio_step5):
    os.makedirs(dir_output_audio_step5)
    
# Create a new directory for processed images if it does not exist
dir_output_images = output_video_path + input_video_filename_wo_ext +'/Images/'
if not os.path.exists(dir_output_images):
    os.makedirs(dir_output_images)
dir_output_images_step1 = output_video_path + input_video_filename_wo_ext +'/Images/Step1_Images/'
if not os.path.exists(dir_output_images_step1):
    os.makedirs(dir_output_images_step1)
dir_output_images_step2 = output_video_path + input_video_filename_wo_ext +'/Images/Step2_Faces/'
if not os.path.exists(dir_output_images_step2):
    os.makedirs(dir_output_images_step2)
dir_output_images_step3 = output_video_path + input_video_filename_wo_ext +'/Images/Step3_Embeddings/'
if not os.path.exists(dir_output_images_step3):
    os.makedirs(dir_output_images_step3)
dir_output_images_step4 = output_video_path + input_video_filename_wo_ext +'/Images/Step4_Clusters/'
if not os.path.exists(dir_output_images_step4):
    os.makedirs(dir_output_images_step4)

# Create a new directory for processed text if it does not exist
dir_output_text = output_video_path + input_video_filename_wo_ext +'/Text/'
if not os.path.exists(dir_output_text):
    os.makedirs(dir_output_text)
dir_output_text_seg = output_video_path + input_video_filename_wo_ext +'/Text/Segments/'
if not os.path.exists(dir_output_text_seg):
    os.makedirs(dir_output_text_seg)

In [ ]:
#####################################################
# Video to Audio
#####################################################

In [ ]:
%%timeit
### Process Audio ###
# Read audio file
sound = AudioSegment.from_file(input_video_filename, 'mp4')

# Stereo to Mono
sound = sound.set_channels(1)

# Sample Rate (must be between 8 kHz and 48 kHz inclusive)
if sound.frame_rate > 48000:
    sound = sound.set_frame_rate(48000)
elif sound.frame_rate < 8000:
    sound = sound.set_frame_rate(8000)

# Normalize Sound
sound = effects.normalize(sound)

# Export processed audio file as Flac
sound.export(dir_output_audio_step1 + 'audio.flac', format='flac')

print('Finish.')

In [ ]:
### Separate background music from vocals ###
separator = Separator('spleeter:2stems')

In [ ]:
%%timeit
### Separate background music from vocals ###
# Load flac using adapter
sound = AudioSegment.from_file(dir_output_audio_step1 + 'audio.flac')
sr = sound.frame_rate
audio_loader = AudioAdapter.default()
signal, _ = audio_loader.load(dir_output_audio_step1 + 'audio.flac', sample_rate=sr)

pred = separator.separate(signal)

# Export processed audio file as Flac
sf.write(dir_output_audio_step2 + 'audio.flac', pred["vocals"], sr)

# separator = None
# gc.collect()
# torch.cuda.empty_cache()
# device = cuda.get_current_device()
# device.reset()
print('Finish.')